# AgentCore Gateway Demo

A Demo on converting a Lambda function into a MCP compatible tool and create an MCP Server exposing the tool with AgentCore Gateway.
- Inbound Auth of the gateway will be controlled by Cognito
- Invoking Gateway with MCP Client Libraries

For more details, please refer to my article [Bedrock AgentCore Part 5: Gateway. Turn Anything(s) into ONE MCP Server in Seconds!]()

## Set Up

In [24]:
import boto3
import json
import zipfile
import io
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
import hmac
import hashlib
import base64
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from datetime import timedelta


region="us-east-1"
lambda_function_name = "agentcore_gateway_target_lambda"
lambda_execution_role_name = "AgentCoreLambdaExecutionRole"
gateway_name = "AgentcoreGateway"
gateway_execution_role_name = "AgentCoreGatewayExecutionRole"
policy_name = "lambda_excess"
gateway_target_name = "LamdaGreetingTarget"

gateway_client = GatewayClient(region_name=region)
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda', region_name = region)
control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region,
    endpoint_url=f"https://bedrock-agentcore-control.{region}.amazonaws.com"
)
cognito_client = boto3.client('cognito-idp', region_name=region)

## Create Lambda (Tool For Gateway)

This lambda will be later used as a Gateway Target:

A target defines the APIs or Lambda function that a Gateway will provide as tools to an agent. Targets can be Lambda functions, OpenAPI specifications, Smithy models, or other tool definitions.

In this demo, we will be creating a simple Lambda function to be used as the target.

#### Lambda Execution Role

In [25]:
response = iam_client.create_role(
    RoleName=lambda_execution_role_name,
    AssumeRolePolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }),
)
lambda_role_arn = response["Role"]["Arn"]
print(f"role created: {lambda_role_arn}")

role created: arn:aws:iam::075198889659:role/AgentCoreLambdaExecutionRole


#### Lambda Function

In [ ]:
# might need to wait for couple seconds before the role above is created succssfully
lambda_code = """
import json
def lambda_handler(event, context):
    return {
        'statusCode': 200,
        'body': json.dumps(f'Hello to {event["name"]}!')
    }
"""

zip_buffer = io.BytesIO()
with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zf:
    zf.writestr('lambda_function.py', lambda_code)
zip_buffer.seek(0)


response = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.13',
    Role=lambda_role_arn,
    Handler='lambda_function.lambda_handler',
    Code={
        'ZipFile': zip_buffer.read()
    },
    Timeout=60,
    MemorySize=128,
    Publish=True,
)
lambda_arn = response["FunctionArn"]
print(f"lambda created: {lambda_arn}")

lambda created: arn:aws:lambda:us-east-1:075198889659:function:agentcore_gateway_target_lambda


## Gateway Authorizer (Inbound Auth)

Since MCP only supports OAuth, each Gateway must have an attached OAuth authorizer. If you don't have an OAuth authorization server already, you will be able to create one in this guide using Cognito.

We Will set up Cognito EZ Auth with AgentCore SDK. This eliminates the complexity of OAuth setup. 

In [27]:
# EZ Auth - automatically sets up Cognito OAuth
# cognito_result: dictionary with details of the authorization server, client id, and client secret
cognito_result = gateway_client.create_oauth_authorizer_with_cognito(gateway_name)
print(json.dumps(cognito_result, indent=2))

2025-08-07 16:48:50,800 - bedrock_agentcore.gateway - INFO - Starting EZ Auth setup: Creating Cognito resources...
2025-08-07 16:48:52,493 - bedrock_agentcore.gateway - INFO -   ✓ Created User Pool: us-east-1_5lxJlsShE
2025-08-07 16:48:53,189 - bedrock_agentcore.gateway - INFO -   ✓ Created domain: agentcore-4c7a7528
2025-08-07 16:48:53,190 - bedrock_agentcore.gateway - INFO -   ⏳ Waiting for domain to be available...
2025-08-07 16:48:53,458 - bedrock_agentcore.gateway - INFO -   ✓ Domain is active
2025-08-07 16:48:53,865 - bedrock_agentcore.gateway - INFO -   ✓ Created resource server: AgentcoreGateway
2025-08-07 16:48:54,337 - bedrock_agentcore.gateway - INFO -   ✓ Created client: 5p10bf31hr5daq32pq6ns1i1bq
2025-08-07 16:48:54,339 - bedrock_agentcore.gateway - INFO -   ⏳ Waiting for DNS propagation of domain: agentcore-4c7a7528.auth.us-east-1.amazoncognito.com
2025-08-07 16:49:54,343 - bedrock_agentcore.gateway - INFO - ✓ EZ Auth setup complete!


{
  "authorizer_config": {
    "customJWTAuthorizer": {
      "allowedClients": [
        "5p10bf31hr5daq32pq6ns1i1bq"
      ],
      "discoveryUrl": "https://cognito-idp.us-east-1.amazonaws.com/us-east-1_5lxJlsShE/.well-known/openid-configuration"
    }
  },
  "client_info": {
    "client_id": "5p10bf31hr5daq32pq6ns1i1bq",
    "client_secret": "1vl0mhupts638i95pk0fivhusqdhvdlid9ordmqvfdmlfvmn2t7f",
    "user_pool_id": "us-east-1_5lxJlsShE",
    "token_endpoint": "https://agentcore-4c7a7528.auth.us-east-1.amazoncognito.com/oauth2/token",
    "scope": "AgentcoreGateway/invoke",
    "domain_prefix": "agentcore-4c7a7528"
  }
}


## Create Gateway

**Semantic search**

Enables intelligent tool discovery so that we are not limited by typical list tools limits (typically 100 or so). This capability delivers contextually relevant tool subsets, significantly improving tool selection accuracy through focused, relevant results, inference performance with reduced token processing and overall orchestration efficiency and response times.



#### Create Gateway Execution Role

In [28]:
response = iam_client.create_role(
    RoleName=gateway_execution_role_name,
    AssumeRolePolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock-agentcore.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }),
)
gateway_role_arn = response["Role"]["Arn"]
print(f"role created: {gateway_role_arn}")

role created: arn:aws:iam::075198889659:role/AgentCoreGatewayExecutionRole


#### MCP Gateway

Currently mcp is the only protocolConfiguration supported.

In [29]:
# gateway: the created Gateway
# Response syntax: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agentcore-control/client/create_gateway.html
gateway = gateway_client.create_mcp_gateway(
    name=gateway_name, # name of the gateway (defaults to TestGateway).
    role_arn=gateway_role_arn, # the role arn to use (creates one if none provided). if not set, one will be created.
    authorizer_config=cognito_result["authorizer_config"], # the authorizer config (will create one if none provided).
    enable_semantic_search=True, # whether to enable search tool (defaults to True).
)
print(f"Gateway created. {gateway["gatewayArn"]}")

2025-08-07 17:03:13,987 - bedrock_agentcore.gateway - INFO - Creating Gateway
2025-08-07 17:03:15,115 - bedrock_agentcore.gateway - INFO - ✓ Created Gateway: arn:aws:bedrock-agentcore:us-east-1:075198889659:gateway/agentcoregateway-svhbi7t2pz
2025-08-07 17:03:15,116 - bedrock_agentcore.gateway - INFO -   Gateway URL: https://agentcoregateway-svhbi7t2pz.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp
2025-08-07 17:03:15,116 - bedrock_agentcore.gateway - INFO -   Waiting for Gateway to be ready...
2025-08-07 17:03:15,421 - bedrock_agentcore.gateway - INFO - 
✅Gateway is ready


Gateway created. arn:aws:bedrock-agentcore:us-east-1:075198889659:gateway/agentcoregateway-svhbi7t2pz


## Add Gateway Target

#### SDK Bug

`gateway_client.create_mcp_gateway_target` will not work if we have a pre-created lambda. We will get the following error.
```
ParamValidationError: Parameter validation failed:
Missing required parameter in input: "credentialProviderConfigurations"
```

**Gateway Target:**

A target defines the APIs or Lambda function that a Gateway will provide as tools to an agent. Targets can be Lambda functions, OpenAPI specifications, Smithy models, or other tool definitions.


Gateway allows us to attach multiple targets to a Gateway and we can change the targets / tools attached to a gateway at any point.

Each target can have its own credential provider attached enabling you to securely access targets whether they need IAM, API Key, or OAuth credentials. Note: the authorization grant flow (three-legged OAuth) is not supported as a target credential type.

With this, Gateway becomes a single MCP URL enabling access to all of the relevant tools for an agent across myriad APIs. Let's dive deeper into how to define each of the target types.

#### Define Target Payload

In [ ]:
# Schema for defining how the gateway uses a Lambda function to communicate with the target,
# as well as the tools provided by the Lambda function.
tool_name = "greeting_tool"
function_schema = [{
    "name": tool_name,
    "description": "Greet the user.",
    "inputSchema": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string"
            }
        },
        "required": ["name"]
    },
    # optional
    "outputSchema": {
        "type": "object",
        "properties": {
            "statusCode": {
                "type": "number"
            },
            "body": {
                "type": "string"
            }
        },
        "required": ["statusCode", "body"]
    }
}]

target_payload = {
    "lambdaArn": lambda_arn,
    "toolSchema": {
        "inlinePayload": function_schema
    }
}

#### Create Target

In [31]:
lambda_target = control_client.create_gateway_target(
    gatewayIdentifier=gateway["gatewayId"],
    name=gateway_target_name,
    targetConfiguration={
        "mcp": { "lambda": target_payload }
    },
    credentialProviderConfigurations=[{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
)

print(f"target created: {lambda_target["targetId"]}")

target created: QU3UWKUXUY


## Outbound Auth / AgentCore Credential Provider

When Gateway makes calls to your APIs or Lambda function it must use some credentials to access those functionalities. When you create a Smithy or Lambda target, Gateway uses the attached execution role to make calls to those targets. 

When you create an OpenAPI target, you must attach an AgentCore credential provider which stores the API Key or OAuth credentials that Gateway will use to access the OpenAPI target. For more details on Setting up Outbound Auth: https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway-outbound-auth.html

#### Add Permission to Gateway Execution Role

In [32]:
# Add policy
iam_client.put_role_policy(
    RoleName=gateway_execution_role_name,
    PolicyName=policy_name,
    PolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "lambda:InvokeFunction",
                ],
                "Resource": lambda_arn
            }
        ]
    })
)
print("Policy added.")

Policy added.


### Add resource-based policy to Lambda

A resource-based policy that allows the Gateway's execution role to invoke it

In [33]:
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='GatewayInvoke',
    Action='lambda:InvokeFunction',
    Principal=gateway_role_arn,
)
print("Policy added to lambda.")

Policy added to lambda.


## Use Gateway with MCP

Gateway implements the Model Context Protocol (MCP), which provides a standardized way for agents to discover and invoke tools. The gateway exposes two main MCP operations:

tools/list: Lists all available tools provided by the gateway
tools/call: Invokes a specific tool with the provided arguments
To use these operations, you need to make HTTP requests to the gateway's MCP endpoint with the appropriate authentication.

#### Making MCP Requests

In [ ]:
# obtain access token
access_token = gateway_client.get_access_token_for_cognito(cognito_result["client_info"])

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token}"
}
url = gateway["gatewayUrl"]

2025-08-07 17:03:43,784 - bedrock_agentcore.gateway - INFO - Fetching test token from Cognito...
2025-08-07 17:03:43,785 - bedrock_agentcore.gateway - INFO -   Attempting to connect to token endpoint: https://agentcore-4c7a7528.auth.us-east-1.amazoncognito.com/oauth2/token
2025-08-07 17:03:44,910 - bedrock_agentcore.gateway - INFO - ✓ Got test token successfully


##### List Tools

In [35]:
target_tool_name = None

async with streamablehttp_client( url=url, headers=headers) as (read_stream, write_stream, callA):
    async with ClientSession(read_stream, write_stream) as session:
        print("Initializing MCP...")
        _init_response = await session.initialize()
        print(f"MCP Server Initialize successful! - {_init_response}")

        # 2. List available tools
        print("Listing tools...")
        cursor = True
        tools = []
        while cursor:
            next_cursor = cursor
            if type(cursor) == bool:
                next_cursor = None
            list_tools_response = await session.list_tools(next_cursor)
            tools.extend(list_tools_response.tools)
            cursor = list_tools_response.nextCursor


        print(f"{len(tools)} tools available")
        for tool in tools:
            if tool_name in tool.name:
                target_tool_name = tool.name
            print(f"Tool: {tool.name}")
            print(f"description: {tool.description}")
            print(f"input schema: {tool.inputSchema}")
            print(f"output schema: {tool.outputSchema}")
            print("---------------")

Initializing MCP...
MCP Server Initialize successful! - meta=None protocolVersion='2025-03-26' capabilities=ServerCapabilities(experimental=None, logging=None, prompts=None, resources=None, tools=ToolsCapability(listChanged=False), completions=None) serverInfo=Implementation(name='AgentcoreGateway', title=None, version='1.0.0') instructions=None
Listing tools...
2 tools available
Tool: x_amz_bedrock_agentcore_search
description: A special tool that returns a trimmed down list of tools given a context. Use this tool only when there are many tools available and you want to get a subset that matches the provided context.
input schema: {'type': 'object', 'properties': {'query': {'type': 'string'}}, 'required': ['query']}
output schema: None
---------------
Tool: LamdaGreetingTarget___greeting_tool
description: Greet the user.
input schema: {'type': 'object', 'properties': {'name': {'type': 'string'}}, 'required': ['name']}
output schema: None
---------------


##### Call Tools

In [18]:
async with streamablehttp_client( url=url, headers=headers) as (read_stream, write_stream, callA):
    async with ClientSession(read_stream, write_stream) as session:
        print("Initializing MCP...")
        _init_response = await session.initialize()
        print(f"MCP Server Initialize successful! - {_init_response}")
        if target_tool_name != None:
            args = {
                "name": "itsuki"
            }
            invoke_tool_response = await session.call_tool(
                target_tool_name,
                arguments=args,
                read_timeout_seconds=timedelta(seconds=60)
            )
            contents = invoke_tool_response.content
            for content in contents:
                text = content.text
                try:
                    content = json.dumps(json.loads(text), indent=2)
                except:
                    content = text
                print(content)


Initializing MCP...
MCP Server Initialize successful! - meta=None protocolVersion='2025-03-26' capabilities=ServerCapabilities(experimental=None, logging=None, prompts=None, resources=None, tools=ToolsCapability(listChanged=False), completions=None) serverInfo=Implementation(name='AgentcoreGateway', title=None, version='1.0.0') instructions=None
{
  "response": {
    "payload": {
      "statusCode": 200,
      "body": "\"Hello to itsuki!\""
    },
    "clientError": false,
    "clientErrorMessage": null,
    "toolInvokeRequestId": "3eec22fc-869e-4dc0-bfc8-05861944ff22"
  }
}


##### Search Tools

Tool provided by AWS for Semantic search: `x_amz_bedrock_agentcore_search`

Input: "query": string

Output: A List of Tool objects that can be used based on the query.  

**Semantic search**

Enables intelligent tool discovery so that we are not limited by typical list tools limits (typically 100 or so). This capability delivers contextually relevant tool subsets, significantly improving tool selection accuracy through focused, relevant results, inference performance with reduced token processing and overall orchestration efficiency and response times.



In [19]:
search_tool_name = "x_amz_bedrock_agentcore_search"
async with streamablehttp_client( url=url, headers=headers) as (read_stream, write_stream, callA):
    async with ClientSession(read_stream, write_stream) as session:
        print("Initializing MCP...")
        _init_response = await session.initialize()
        print(f"MCP Server Initialize successful! - {_init_response}")
        args = {
            "query": "I want to say hello to itsuki"
        }
        invoke_tool_response = await session.call_tool(
            search_tool_name,
            arguments=args,
            read_timeout_seconds=timedelta(seconds=60)
        )
        contents = invoke_tool_response.content
        for content in contents:
            text = content.text
            try:
                content = json.dumps(json.loads(text), indent=2)
            except:
                content = text
            print(content)

Initializing MCP...
MCP Server Initialize successful! - meta=None protocolVersion='2025-03-26' capabilities=ServerCapabilities(experimental=None, logging=None, prompts=None, resources=None, tools=ToolsCapability(listChanged=False), completions=None) serverInfo=Implementation(name='AgentcoreGateway', title=None, version='1.0.0') instructions=None
{
  "tools": [
    {
      "inputSchema": {
        "type": "object",
        "properties": {
          "name": {
            "type": "string"
          }
        },
        "required": [
          "name"
        ]
      },
      "name": "LamdaGreetingTarget___greeting_tool",
      "description": "Greet the user."
    }
  ]
}


## Clean Up

In [38]:
# delete gateway target
response = control_client.delete_gateway_target(
    gatewayIdentifier=gateway["gatewayId"],
    targetId = lambda_target["targetId"]
)

# delete gateway
response = control_client.delete_gateway(
    gatewayIdentifier=gateway["gatewayId"]
)

In [39]:
user_pool_id = cognito_result["client_info"]["user_pool_id"]
describe_response = cognito_client.describe_user_pool(
    UserPoolId=user_pool_id,
)
user_pool_config = describe_response["UserPool"]
# delete domain
response = cognito_client.delete_user_pool_domain(
    Domain=user_pool_config["Domain"],
    UserPoolId=user_pool_id
)
# delete user pool
response = cognito_client.delete_user_pool(
    UserPoolId=user_pool_id
)

In [40]:
# delete lambda
response = lambda_client.delete_function(
    FunctionName=lambda_function_name,
)

In [41]:
# delete role policy and role
iam_client.delete_role_policy(
    RoleName=gateway_execution_role_name,
    PolicyName=policy_name
)

iam_client.delete_role(
    RoleName = gateway_execution_role_name
)

iam_client.delete_role(
    RoleName = lambda_execution_role_name
)

print("Deleted!")

Deleted!
